<img src="../images/ARCO.jpeg" width=800 alt="Project Pythia Logo"></img>

# Analysis-Ready Cloud-Optimized Datasets

---

## Overview
In this notebook, we will explore Analysis-Ready Cloud-Optimized (ARCO) radar datasets using Canadian weather radar data. You'll learn:

1. **Analysis-Ready datasets** - Pre-processed data ready for immediate analysis
2. **Cloud-Optimized formats** - Efficient storage and access in cloud environments  
3. **FAIR principles** - Making data Findable, Accessible, Interoperable, and Reusable
4. **Zarr format** - Modern chunked storage for large scientific datasets

We'll use Canadian radar data from the May 2022 Ontario Derecho severe weather event.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray) | Necessary |  Basic features |
| [Radar Cookbook](https://projectpythia.org/radar-cookbook/) | Necessary |  Radar basics |
|[Intro to Zarr](https://zarr.readthedocs.io/en/stable/tutorial.html)| Necessary | Zarr basics

- **Time to learn**: 30 minutes


---

## Imports


## Analysis-Ready Data

**Analysis-Ready data** means datasets are prepared and structured to be immediately usable for scientific analysis. Studies show that data scientists typically spend ~80% of their time preparing and cleaning data rather than doing actual analysis.

Analysis-Ready datasets solve this by providing:
- **Clean, pre-processed data** that's ready to use
- **Rich metadata** that explains what the data contains
- **Standardized formats** that work well with analysis tools
- **Quality control** that ensures data reliability

This means more time for science and discovery! 🚀

<img src="../images/AR.jpeg" width=400 alt="Project Pythia Logo"></img>

**Key Benefits of Analysis-Ready Data:**

✅ **Datasets instead of scattered files** - Organized collections of related data  
✅ **Pre-processed and clean** - No need to spend hours fixing data issues  
✅ **Rich metadata included** - Clear documentation of what the data represents  
✅ **Cataloged and discoverable** - Easy to find relevant datasets  
✅ **Immediate analysis capability** - Start analyzing right away  
✅ **More time for science!** - Focus on research questions, not data wrangling

## Cloud-Optimized Data

Traditional radar data formats (like individual NetCDF files) work well on local computers but are slow and inefficient in cloud environments. **Cloud-Optimized** formats like Zarr are designed specifically for fast, efficient access from cloud storage.

![Move to cloud diagram](../images/cloud-move.png)

**Why Cloud-Optimized matters:**
- **Parallel access** - Multiple users can read different parts simultaneously
- **Chunked storage** - Only download the data you need
- **Fast streaming** - No need to download entire files
- **Scalable processing** - Handle datasets too large for local computers

## FAIR Data Principles

**FAIR** data follows principles that make scientific data more valuable and reusable:

- **F**indable - Easy to discover through catalogs and search
- **A**ccessible - Available through standard protocols  
- **I**nteroperable - Works with different tools and systems
- **R**eusable - Well-documented for future use by others

<!-- ![fair](../images/fair-data-principles.jpg) -->
<img src="../images/fair-data-principles.jpg" width=600 alt="Fair"></img>

**FAIR data benefits everyone:**
- **Data producers** get citations when others use their datasets
- **Data consumers** access interesting datasets that would otherwise be unavailable
- **Science advances** through improved data sharing and collaboration

<img src="../images/fair-reuse-300dpi.png" width=400 alt="Fair"></img>

*Image courtesy: [Zarr illustrations](https://github.com/zarr-developers/zarr-illustrations-falk-2022)*

## Zarr format

**Zarr** is a modern storage format designed for large scientific datasets. Instead of storing data in single large files, Zarr breaks data into small "chunks" that can be:

- **Compressed** to save storage space
- **Accessed in parallel** by multiple users
- **Streamed efficiently** from cloud storage
- **Processed on-demand** without downloading everything

Think of it like having a library where you can grab just the books you need, rather than having to check out the entire library!

<img src="../images/monolithic-vs-chunked-1200dpi.png" width=200 alt="zarr"></img>

Courtesy: [Zarr illustrations](https://github.com/zarr-developers/zarr-illustrations-falk-2022?tab=readme-ov-file)

## Creating ARCO Radar Datasets

We'll create Analysis-Ready Cloud-Optimized radar datasets using the **CfRadial2.1/FM301 standard** - a hierarchical structure endorsed by the World Meteorological Organization (WMO). This standard organizes radar data efficiently for both storage and analysis.

### CfRadial2.1/FM301 standard

The **DataTree structure** organizes radar data hierarchically:
- **Root level**: Contains general radar metadata (location, time, etc.)
- **Sweep levels**: Each elevation angle gets its own dataset with radar variables
- **This structure** mirrors how meteorologists think about radar scans

<img src="../images/CfRadial2.1.svg" width=400 alt="xradar"></img>

Let's explore how this works with Canadian radar data from the Ontario Derecho event

In [12]:
# Configure S3 storage for icechunk
storage = icechunk.s3_storage(
    bucket='pythia',
    prefix='radar/ams2025/CASSM.zarr',
    endpoint_url='https://js2.jetstream-cloud.org:8001',
    anonymous=True,
    region='us-east-1',
    force_path_style=True
)

In [13]:
repo = icechunk.Repository.open(storage=storage)

In [14]:
session= repo.readonly_session("main")

In [16]:
dtree = xr.open_datatree(
    session.store, 
    engine="zarr",
    consolidated=False,
    chunks={}
)

In [17]:
dtree

<xarray.DataTree>
Group: /
├── Group: /sweep_15
│       Dimensions:            (vcp_time: 56, azimuth: 720, range: 660)
│       Coordinates:
│           elevation          (azimuth) float64 6kB dask.array<chunksize=(720,), meta=np.ndarray>
│           latitude           float64 8B ...
│           altitude           float64 8B ...
│         * range              (range) float32 3kB 250.0 750.0 ... 3.292e+05 3.298e+05
│           crs_wkt            int64 8B ...
│         * azimuth            (azimuth) float64 6kB 0.25 0.75 1.25 ... 359.2 359.8
│           longitude          float64 8B ...
│         * vcp_time           (vcp_time) datetime64[ns] 448B 2024-08-05T23:54:03 ......
│           y                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│           time               (azimuth) datetime64[ns] 6kB dask.array<chunksize=(720,), meta=np.ndarray>
│           x                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│           z                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│       Data variables:
│           DBZH               (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           PHIDP              (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           KDP                (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           VRADH              (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           UPHIDP             (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           ZDR                (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           TH                 (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           RHOHV              (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           SQIH               (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           WRADH              (vcp_time, azimuth, range) float32 106MB dask.array<chunksize=(1, 720, 660), meta=np.ndarray>
│           sweep_fixed_angle  (vcp_time) float32 224B dask.array<chunksize=(1,), meta=np.ndarray>
│           sweep_number       (vcp_time) float64 448B dask.array<chunksize=(1,), meta=np.ndarray>
├── Group: /georeferencing_correction
│       Dimensions:    (vcp_time: 56)
│       Coordinates:
│           altitude   float64 8B ...
│           longitude  float64 8B ...
│           latitude   float64 8B ...
│         * vcp_time   (vcp_time) datetime64[ns] 448B 2024-08-05T23:54:03 ... 2024-08...
├── Group: /sweep_13
│       Dimensions:            (vcp_time: 56, azimuth: 720, range: 660)
│       Coordinates:
│           latitude           float64 8B ...
│         * azimuth            (azimuth) float64 6kB 0.25 0.75 1.25 ... 359.2 359.8
│           time               (azimuth) datetime64[ns] 6kB dask.array<chunksize=(720,), meta=np.ndarray>
│         * vcp_time           (vcp_time) datetime64[ns] 448B 2024-08-05T23:54:03 ......
│           longitude          float64 8B ...
│         * range              (range) float32 3kB 250.0 750.0 ... 3.292e+05 3.298e+05
│           elevation          (azimuth) float64 6kB dask.array<chunksize=(720,), meta=np.ndarray>
│           altitude           float64 8B ...
│           z                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│           y                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│           crs_wkt            int64 8B ...
│           x                  (azimuth, range) float64 4MB dask.array<chunksize=(180, 330), meta=np.ndarray>
│       Data variab

---

## Summary

We successfully created **Analysis-Ready Cloud-Optimized (ARCO)** radar datasets using Canadian weather data! Here's what we accomplished:

### 🎯 **Key Learning Outcomes:**

**📊 Analysis-Ready**: Pre-processed, clean datasets ready for immediate scientific analysis  
**☁️ Cloud-Optimized**: Efficient Zarr format enabling fast access from cloud storage  
**🌐 FAIR Principles**: Making data Findable, Accessible, Interoperable, and Reusable  
**📈 Time Series**: Combined multiple radar volumes to track storm evolution  
**🏗️ Standardized Structure**: Used WMO-endorsed FM301 hierarchical organization

### 🚀 **What This Enables:**
- **Faster Research**: No more data preprocessing - start analyzing immediately
- **Cloud Analytics**: Process large datasets without downloading everything  
- **Reproducible Science**: Standardized formats work across different tools
- **Collaboration**: Easy data sharing following FAIR principles
- **Storm Tracking**: Time series analysis of severe weather events

The Ontario Derecho case study demonstrates how ARCO datasets streamline radar meteorology research and education! 🌪️

## Resources and references
 - [Xradar](https://docs.openradarscience.org/projects/xradar/en/stable/index.html)
 - [Radar cookbook](https://github.com/ProjectPythia/radar-cookbook)
 - [Py-Art landing page](https://arm-doe.github.io/pyart/)
 - [Wradlib landing page](https://docs.wradlib.org/en/latest/index.html)